DTM 내에 있는 각 단어에 대한 중요도를 계산할 수 있는 TF-IDF 가중치에 대해 다뤄보자. 이걸 사용하면 기존의 DTM을 사용하는 것보다 많은 정보를 고려해 문서를 비교할 수 있다. 

# 1. TF-IDF(단어 빈도-역문서 빈도)

TF-IDF(Term Frequency-Inverse Document Frequency)는 단어의 빈도와 역 문서 빈도(문서의 빈도에 특정 식을 취함)를 사용하여 DTM 내의 각 단어들마다 중요한 정도를 가중치로 주는 방법입니다. 우선 DTM을 만든 후, TF-IDF 가중치를 부여합니다.

TF-IDF는 주로 문서의 유사도를 구하는 작업, 검색 시스템에서 검색 결과의 중요도를 정하는 작업, 문서 내에서 특정 단어의 중요도를 구하는 작업 등에 쓰일 수 있습니다.

TF-IDF는 모든 문서에서 자주 등장하는 단어는 중요도가 낮다고 판단하며, 특정 문서에서만 자주 등장하는 단어는 중요도가 높다고 판단합니다. TF-IDF 값이 낮으면 중요도가 낮은 것이며, TF-IDF 값이 크면 중요도가 큰 것입니다. 즉, the나 a와 같이 불용어의 경우에는 모든 문서에 자주 등장하기 마련이기 때문에 자연스럽게 불용어의 TF-IDF의 값은 다른 단어의 TF-IDF에 비해서 낮아지게 됩니다

# 2. 파이썬으로 TF-IDF 직접 구현


In [3]:
import pandas as pd
from math import log#IDF 구현

docs=[
  '먹고 싶은 사과',
  '먹고 싶은 바나나',
  '길고 노란 바나나 바나나',
  '저는 과일이 좋아요'
]

vocab=list(set(w for doc in docs for w in doc.split()))
vocab.sort()

In [4]:
vocab

['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']

In [2]:
#TF,IDF 그리고 TF-IDF 값을 구하는 함수 구현

N=len(docs)

def tf(t,d):#문서안에 단어가 몇개
  return d.count(t)

def idf(t):
  df=0
  for doc in docs:
    df+=t in doc
  return log(N/(df+1))

def tfidf(t,d):
  return tf(t,d)*idf(t)

TF를 구해보겠다. DTM을 데이터 프레임에 저장해 출력해보겠다. 

In [6]:
result=[]

for i in range(N):
  result.append([])
  d=docs[i]
  for j in range(len(vocab)):
    t=vocab[j]
    result[-1].append(tf(t,d))

tf_=pd.DataFrame(result,columns=vocab)

In [7]:
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


정상적으로 DTM이 출력되었다. 각단어에 대한 IDF 값을 구해보자 .

In [8]:
result=[]

for j in range(len(vocab)):
  t=vocab[j]
  result.append(idf(t))

idf_=pd.DataFrame(result,index=vocab,columns=["IDF"])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [9]:
result=[]

for i in range(N):
  result.append([])
  d=docs[i]
  for j in range(len(vocab)):
    t=vocab[j]
    result[-1].append(tfidf(t,d))

tfidf_=pd.DataFrame(result,columns=vocab)
tfidf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


# 3. 사이킷런을 이용한 DTM과 TF-IDF 실습

BoW를 설명하며 배운 CountVectorizer를 사용하면 DTM을 만들 수 있다. 

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

corpus=[
    'you know I want your love',
    'I like you',
    'what should I do ',    
]

vector=CountVectorizer()

#코퍼스로부터 각 단어의 빈도수 기록
print(vector.fit_transform(corpus).toarray())

#각 단어와 매핑된 인덱스 출력
print(vector.vocabulary_)

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


사이킷런은 TF-IDF를 자동 계산해주는 TfidfVectorizer를 제공합니다. 사이킷런의 TF-IDF는 위에서 배웠던 보편적인 TF-IDF 기본 식에서 조정된 식을 사용합니다. 요약하자면, IDF의 로그항의 분자에 1을 더해주며, 로그항에 1을 더해주고, TF-IDF에 L2 정규화라는 방법으로 값을 조정하는 등의 차이로 TF-IDF가 가진 의도는 여전히 그대로 갖고 있습니다.

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus=[
    'you know I want your love',
    'I like you',
    'what should I do ',  
]

tfidfv=TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


BoW, DTM, TF-IDF에 대해서 전부 학습했습니다. 문서들 간의 유사도를 구하기 위한 재료 손질하는 방법을 배운 셈입니다. 케라스로도 DTM과 TF-IDF 행렬을 만들 수 있는데, 이는 딥 러닝 챕터의 다층 퍼셉트론으로 텍스트 분류하기 실습에서 별도로 다루겠습니다. 다음 챕터에서 유사도를 구하는 방법과 이를 이용한 실습을 진행해보겠습니다.